In [1]:
from langchain_community.document_loaders import TextLoader # Take raw text of descriptions and convert into format langchain can work with

from langchain_text_splitters import CharacterTextSplitter # Split whole document containing descriptions into meaningful chunks
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from NLP.Book_Recommendation_Project.LangChain_Top_K import text_splitter

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pandas as pd
movies = pd.read_csv('movies-extended.csv')

In [4]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama


In [5]:
movies['movie_id'] = movies.index

In [6]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre,movie_id
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN,0
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN,1
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN,2
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN,3
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical,22341
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama,22342
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN,22343
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama,22344


In [15]:
movies['id-plot'] = movies['movie_id'].astype(str) + ' ' + movies['Plot']

In [16]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre,movie_id,id-plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN,0,"0 A bartender is working at a saloon, serving ..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN,1,"1 The moon, painted with a smiling face hangs ..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN,2,"2 The film, just over a minute long, is compos..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN,3,3 Lasting just 61 seconds and consisting of tw...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN,4,4 The earliest known adaptation of the classic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical,22341,22341 One night on the campus of McGill Univer...
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama,22342,"22342 The Indian Horse family, including six-y..."
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN,22343,"22343 In rural 1930s Quebec, Alice lives in ho..."
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama,22344,"22344 Opened by Mandarin theme song, Meditatio..."


In [24]:
movies['id-plot'].to_csv('movies_semantic.txt', index=False, header=False)

In [25]:
raw_docs = TextLoader('movies_semantic.csv').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator='\n')
documents = text_splitter.split_documents(raw_docs)

Created a chunk of size 2148, which is longer than the specified 1000
Created a chunk of size 1917, which is longer than the specified 1000
Created a chunk of size 3782, which is longer than the specified 1000
Created a chunk of size 3117, which is longer than the specified 1000
Created a chunk of size 1181, which is longer than the specified 1000
Created a chunk of size 1165, which is longer than the specified 1000
Created a chunk of size 1167, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000
Created a chunk of size 1116, which is longer than the specified 1000
Created a chunk of size 2547, which is longer than the specified 1000
Created a chunk of size 1116, which is longer than the specified 1000
Created a chunk of size 2520, which is longer than the specified 1000
Created a chunk of size 1116, which is longer than the specified 1000
Created a chunk of size 2525, which is longer than the specified 1000
Created a chunk of s

KeyboardInterrupt: 

In [23]:
documents[0]

Document(metadata={'source': 'movies_semantic.csv'}, page_content='1901,Kansas Saloon Smashers,American,Unknown,,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Smashers,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man\'s bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation\'s face before a group of policemen appear and order everybody to leave.[1]","Kansas Saloon Smashers : A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man\'s bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin 

In [18]:
def retrieve_semantic_recommendations(query, top_k=10):
    recs = movies.similarity_search(query, k=top_k)

    movies_list = []

    for i in range(0, len(recs)):
        id = int(recs[i].page_content.strip('"').split()[0])
        movies_list.append(id)

    return movies[movies["movie_id"].isin(movies_list)]